In [ ]:
!pip install -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

In [ ]:
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer
)
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()

In [ ]:
huggingface_dataset_name = "yasminesarraj/texts_summary"

dataset = load_dataset(huggingface_dataset_name)
print (dataset)

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
    use_fast=False
    )
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [ ]:
def gen(model,p, maxlen=1000, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"),
                         max_new_tokens=maxlen,
                         pad_token_id=eval_tokenizer.eos_token_id,
                         do_sample=sample,
                         num_return_sequences=1,
                         temperature=0.1,
                         num_beams=1,
                         top_p=0.95,
                         ).to('cpu')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(
    base_model,
    "/content/drive/MyDrive/checkpoint-400",
    torch_dtype=torch.float16,
    is_trainable=False
    )

In [ ]:
%%time
import pandas as pd

txt_list = dataset['test'][17:21]['text']
summaries = dataset['test'][17:21]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []


for idx, text in enumerate(txt_list):
  txt = txt_list[idx]
  summary = summaries[idx]

  prompt = f"Instruct: Summarize the following scientific reports .\n{txt}\nOutput:\n"


  peft_model_res = gen(ft_model, prompt, 100,)
  peft_model_output = peft_model_res[0].split('Output:\n')[1]
  peft_model_text_output, success, result = peft_model_output.partition('#End')


  original_model_res = gen(base_model, prompt, 100,)
  original_model_text_output = original_model_res[0].split('Output:\n')[1]

  original_model_summaries.append(original_model_text_output)
  peft_model_summaries.append(peft_model_text_output)
  instruct_model_summaries.append(summary)


zipped_summaries = list(zip(instruct_model_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])

df



In [ ]:
import evaluate

rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=original_model_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=original_model_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

In [ ]:
#model = ft_model.merge_and_unload()
#model.save_pretrained("merged_adapters")